In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute_target import ComputeTargetException
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacityworkspace
Azure region: canadacentral
Subscription id: e9204f07-b8a0-4dfa-b8ca-153a9a4abda8
Resource group: udacity_course


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "udacity-project"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.core import ScriptRunConfig, Environment
env = Environment.get(workspace=ws, name="AzureML-Tutorial")

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C":choice(1,3),
    "--max_iter":choice(100,1000)
})


# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")


est = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config= est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

In [5]:

# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project1,HD_58477973-0910-45f7-8e01-151ad83664b1,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [13]:
hyperdrive_run.get_status()

'Completed'

In [7]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [21]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\nAccuracy:', best_run_metrics['Accuracy'])
print('\n')
print(best_run_metrics)
print('\n')
print(best_run)

Exception: No best run was found

In [22]:
best_run.get_file_names()

AttributeError: 'NoneType' object has no attribute 'get_file_names'

In [ ]:
model = best_run.register_model(model_name='best-model-hdr', model_path='outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

dataset_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data = TabularDatasetFactory.from_delimited_files(path=dataset_path)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###